Useful reading:  
The illustrated transformer: http://jalammar.github.io/illustrated-transformer/  
Attention is all you need: https://arxiv.org/pdf/1706.03762.pdf  
BERT: https://arxiv.org/pdf/1810.04805.pdf  
A Visual Guide to Using BERT for the First Time: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/  
Tensorflow hub: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2

In [50]:
import pandas as pd
from bert import tokenization
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
from sklearn import model_selection
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [51]:
train_df = pd.read_csv("nlp-getting-started/train.csv")
test_df = pd.read_csv("nlp-getting-started/test.csv")

In [52]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}


In [53]:
def replace(match):
    return contractions[match.group(0)]

def expand_contractions(s):
    pattern = "|".join(contractions.keys())
    return re.sub(pattern, replace, s)

def remove_usernames(s):
    pattern = "(@[A-Za-z0-9\_]+)"
    return re.sub(pattern, " ", s)

def remove_links(s):
    pattern = "http[\S]+"
    return re.sub(pattern, "", s)

def remove_emojis(s):
    pattern = "\[" "\U0001F600-\U0001F64F" 
    "\U0001F300-\U0001F5FF" 
    "\U0001F680-\U0001F6FF" 
    "\U0001F1E0-\U0001F1FF""\]+"
    return re.sub(pattern, "", s)

def remove_hashtags(s):
    pattern = "(#[A-Za-z0-9\_]+)"
    return re.sub(pattern, "", s)


In [54]:
train_df['text'] = train_df['text'].astype(str)
train_df['text'] = train_df['text'].apply(expand_contractions)
train_df['text'] = train_df['text'].apply(remove_usernames)
train_df['text'] = train_df['text'].apply(remove_links)
train_df['text'] = train_df['text'].apply(remove_emojis)
train_df['text'] = train_df['text'].apply(remove_hashtags)
#print(train_df['text'])

In [71]:
keyword_data = train_df['keyword']

keyword_data.isnull().sum()
unique_keywords = keyword_data.unique().tolist() 

keyword_codes, keyword_uniques = keyword_data.factorize()
keyword_dummies = keyword_data.str.get_dummies() # one hot encoding 

In [59]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable = False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [4]:
def encode_text(text_input):
    input_ids = []
    sequence_ids = []
    attention_mask_ids = []
    for text in text_input:
        input_tokens = tokenizer.tokenize(text)
        input_tokens = ['[CLS]'] + input_tokens + ['[SEP]']
        padding_len = max_len - len(input_tokens)
        token_ids = tokenizer.convert_tokens_to_ids(input_tokens)
        token_ids += [0] * padding_len
        attention_mask = [1] * len(input_tokens) + [0] * padding_len
        sequence = [1] * max_len

        attention_mask_ids.append(attention_mask)
        sequence_ids.append(sequence)
        input_ids.append(token_ids)
    
    sequence_ids = np.array(sequence_ids)
    input_ids = np.array(input_ids)
    attention_mask_ids = np.array(attention_mask_ids)
    return input_ids, attention_mask_ids, sequence_ids

In [5]:
max_len = 512
labels = np.array(train_df.target)
input_data = encode_text(train_df.text)
test_data = encode_text(test_df.text)

NameError: name 'tokenizer' is not defined

In [ ]:
max_seq_length = 512
input_tokens_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
attention_mask_ids= Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask") 
sequence_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
inputs = [input_tokens_ids, attention_mask_ids, sequence_ids]
pooled_output, sequence_output = bert_layer(inputs)
h = Dense(256, activation = "relu")(sequence_output[:, 0, :])
h = Dense(256, activation = "relu")(h)
outputs = Dense(1, activation = 'sigmoid')(h)
model = Model(inputs=inputs, outputs=outputs)
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = 'accuracy')

In [ ]:
checkpoint_file_path = "model_checkpoint_weights/ weights.{epoch:02d}-{val_loss:.2f}.h5"
model_checkpoint = ModelCheckpoint(filepath = checkpoint_file_path, 
                                   save_weights_only = True,
                                   monitor = 'val_acc', 
                                   mode = 'max',
                                   save_best_only = True,
                                   verbose = True)

early_stopping = EarlyStopping(monitor="val_loss",
                                min_delta=0.01,
                                patience=3,
                                verbose=1,
                                mode="auto",
                                baseline=None,
                                restore_best_weights=True)

In [ ]:
model.fit(input_data, labels, epochs = 10, batch_size = 32, validation_split = 0.3, callbacks = [model_checkpoint, early_stopping])

In [ ]:
weight_file = ""
model.load_weights(weight_file)
pred = model.predict(test_input)

In [ ]:
kaggle_submission = pd.DataFrame(columns = ['target'])
kaggle_submission['target'] = pred.round().astype(int)
kaggle_submission.to_csv('kaggle_submission.csv', index = True)